<a href="https://colab.research.google.com/github/safaksonmezz/Multi-type_vehicles_flow_statistics/blob/master/word_embeddings_w2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete89YSZL/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete89YSZL/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [6]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [12]:
training_labels = []
training_sentences = []

testing_labels = []
testing_sentences = []

for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [15]:
len(training_labels)

25000

In [20]:
len(testing_sentences)

25000

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)


testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)

In [23]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [27]:
num_epochs = 10
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)
model.fit(padded,
          training_labels_final,
          epochs = num_epochs,
          validation_data = (testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 6s 7ms/step - loss: 0.5061 - accuracy: 0.7297 - val_loss: 0.3479 - val_accuracy: 0.8482
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2410 - accuracy: 0.9069 - val_loss: 0.3762 - val_accuracy: 0.8339
Epoch 3/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0876 - accuracy: 0.9782 - val_loss: 0.4520 - val_accuracy: 0.8262
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0207 - accuracy: 0.9974 - val_loss: 0.5305 - val_accuracy: 0.8258
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0046 - accuracy: 0.9999 - val_loss: 0.5894 - val_accuracy: 0.8266
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.6378 - val_accuracy: 0.8272
Epoch 7/10
782/782 [==============================] - 5s 7ms/step - loss: 7.7066e-04 - accuracy: 1.0000 - val_loss: 0.6748 - val_accuracy: 0.8286
Ep

In [29]:
import pandas as pd
data_df = pd.read_json("/content/Sarcasm_Headlines_Dataset_v2.json", lines = True)

In [30]:
data_df

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...
...,...,...,...
28614,1,jews to celebrate rosh hashasha or something,https://www.theonion.com/jews-to-celebrate-ros...
28615,1,internal affairs investigator disappointed con...,https://local.theonion.com/internal-affairs-in...
28616,0,the most beautiful acceptance speech this week...,https://www.huffingtonpost.com/entry/andrew-ah...
28617,1,mars probe destroyed by orbiting spielberg-gat...,https://www.theonion.com/mars-probe-destroyed-...


In [31]:
sentences = list(data_df['headline'].values)
labels = np.array(data_df['is_sarcastic'].values)

In [34]:
training_sentences, training_labels, testing_sentences, testing_labels = sentences[:20000], labels[:20000], sentences[20000:], labels[20000:]

In [37]:
vocab_size = 10000
embedding_dim = 16
max_length = 32
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
training_sequences = tokenizer.texts_to_sequences(training_sentences)
padded_training = pad_sequences(training_sequences, maxlen= max_length, truncating= trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
padded_testing = pad_sequences(testing_sequences, maxlen= max_length)

In [40]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
num_epochs = 10
training_labels_final = training_labels
testing_labels_final = testing_labels
model.fit(padded_training,
          training_labels_final,
          epochs = num_epochs,
          validation_data = (padded_testing, testing_labels_final))


Epoch 1/10
625/625 [==============================] - 4s 6ms/step - loss: 0.4826 - accuracy: 0.7591 - val_loss: 0.3355 - val_accuracy: 0.8553
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2357 - accuracy: 0.9078 - val_loss: 0.3185 - val_accuracy: 0.8630
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1286 - accuracy: 0.9560 - val_loss: 0.3590 - val_accuracy: 0.8493
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0596 - accuracy: 0.9847 - val_loss: 0.4235 - val_accuracy: 0.8429
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0258 - accuracy: 0.9954 - val_loss: 0.4798 - val_accuracy: 0.8421
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0117 - accuracy: 0.9984 - val_loss: 0.5353 - val_accuracy: 0.8407
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0056 - accuracy: 0.9994 - val_loss: 0.5874 - val_accuracy: 0.8395
Epoch 